In [1]:
import json
import numpy as np
import pandas as pd
import re

In [2]:
def get_list_of_dicts(fname): return [json.loads(i) for i in open(fname, "rt")]

In [3]:
raw_data = get_list_of_dicts("../data/reviews_Amazon_Instant_Video_5.json")
raw_data[0]

{'asin': 'B000H00VBQ',
 'helpful': [0, 0],
 'overall': 2.0,
 'reviewText': "I had big expectations because I love English TV, in particular Investigative and detective stuff but this guy is really boring. It didn't appeal to me at all.",
 'reviewTime': '05 3, 2014',
 'reviewerID': 'A11N155CW1UV02',
 'reviewerName': 'AdrianaM',
 'summary': 'A little bit boring for me',
 'unixReviewTime': 1399075200}

In [5]:
from IPython.display import display
data = pd.DataFrame(raw_data).loc[:, ["reviewerID", "reviewText", "asin", "overall"]]
user_item_revew = data.drop("reviewText", axis=1)
user_reviews = pd.pivot_table(data, index=["reviewerID", "asin"], aggfunc=lambda x: x).drop("overall", axis=1)
movie_reviews = pd.pivot_table(data, index=["asin", "reviewerID"], aggfunc=lambda x: x).drop("overall", axis=1)

In [6]:
def add_user_reviews(x):
    ur = user_reviews.loc[x["reviewerID"]].drop(x["asin"])
    mr = movie_reviews.loc[x["asin"]].drop(x["reviewerID"])
    x["userReviews"] = ur["reviewText"].tolist()
    x["movieReviews"] = mr["reviewText"].tolist()
    return x
dat = user_item_revew.apply(add_user_reviews, axis=1)

In [7]:
dat.loc[0, ["userReviews"]][0]

['I really like the characters and the actors. It is a different, unconventional approach of the encounters of two people that are lost in life, philosophically speaking.THis movie cheered me up. Perfect for when you are in the mood for something mellow and uncomplicated.',
 "Not much at least, but I didn't get interested in finish it so I guess that says a lot about the movie. The characters didn't make it for me, kind of boring.",
 "I should watch all the episodes. Still I didn't. I like the actors very much especially Mads. He is great and has so a particular and odd face that it is perfect for the series but still I didn't see enough. If i have to say something now I think that could be better done, it is lacking something but I don't know what.",
 'It was so boring I have to pass it forward. Although probably it is a market for this kind of movie.',
 'I am not sure about this one yet. I have to see more. The problem is lately all these kinds of shows look alike and is very hard to

In [12]:
dat.to_csv(path_or_buf="../data/unembedded_grouped_reviews.csv")